In [2]:
import numpy as np
import pandas as pd

In [3]:
want_new_data = True

if want_new_data:
    data = pd.read_pickle('crafted_data/playlists_and_trackid_start0_end6900.pkl')
    data['num_tracks_id'] = data['tracks'].apply(lambda row: len(row))
    data = data[data['num_tracks_id'] >= 10]
    max_length = max(data['num_tracks_id'])

    numerical_values = pd.read_pickle('src/tracks_numerical_features.pkl')
    numerical_values = numerical_values.set_index('id')
    X = []
    y = []
    for playlist in data['tracks']:
        curr = []
        for i in range(len(playlist)):
            
            song_id = playlist[i]
            if (i == len(playlist) - 1):
                y.append(np.array(numerical_values.loc[song_id].values))
            else:
                curr.append(np.array(numerical_values.loc[song_id].values))
        to_pad = max_length - len(curr) - 1
        tmp = np.array(curr)
        tmp = np.pad(tmp, pad_width=((to_pad,0), (0,0)), mode='constant')
        X.append(np.array(tmp))
    X = np.array(X)
    y = np.array(y)
    np.save('crafted_data/X', X)
    np.save('crafted_data/y', y)
else:
    X = np.load('crafted_data/X.npy')
    y = np.load('crafted_data/y.npy')

print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (3566, 120, 13)
y shape:  (3566, 13)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (2852, 120, 13)
Training Labels Shape: (2852, 13)
Testing Features Shape: (714, 120, 13)
Testing Labels Shape: (714, 13)


In [5]:
print(numerical_values.columns)
['explicit', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature']
songs = [f'song{i}' for i in range(1,max_length)]
column_names = []
for song in songs:
    column_names.append([f'{song}_explicit', f'{song}_danceability', f'{song}_energy', f'{song}_key', f'{song}_loudness', f'{song}_mode',
       f'{song}_speechiness', f'{song}_acousticness', f'{song}_instrumentalness', f'{song}_liveness',
       f'{song}_valence', f'{song}_tempo', f'{song}_time_signature'])


column_names = np.array(column_names).flatten()
column_names = np.append(column_names, ['next_song_danceability'])
dataset = pd.DataFrame({}, columns=column_names)
for i in range(X.shape[0]):
    dataset.loc[len(dataset)] = np.append(X[i].flatten(), y[i,1])
dataset.shape

Index(['explicit', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature'],
      dtype='object')


(3566, 1561)

In [14]:
def split_dataset(dataset, test_ratio=0.25):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset)
print("{} examples in training, {} examples for testing.".format(len(train_ds_pd), len(test_ds_pd)))


2719 examples in training, 847 examples for testing.


In [19]:
import tensorflow_decision_forests as tfdf
from tensorflow.keras.metrics import CosineSimilarity


label = 'next_song_danceability'
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)

# model.compile(metrics=[CosineSimilarity(axis=1)])
model.compile(metrics=['mse', CosineSimilarity(axis=1)])
model.fit(train_ds);
print(model.summary());

/Users/zariski/mambaforge/envs/ml/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features_dataframe = dataframe.drop(label, 1)
/Users/zariski/mambaforge/envs/ml/lib/python3.10/site-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  features_dataframe = dataframe.drop(label, 1)


Use /var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/tmpslpq9ite as temporary training directory
Reading training dataset...


Training dataset read in 0:00:08.258859. Found 2719 examples.
Training model...


[INFO kernel.cc:1176] Loading model from path /var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/tmpslpq9ite/model/ with prefix 9aa59b8faacd4727


Model trained in 0:00:24.586041
Compiling model...


[INFO decision_forest.cc:639] Model loaded with 300 root(s), 206112 node(s), and 657 input feature(s).
[INFO abstract_model.cc:1249] Engine "RandomForestOptPred" built
[INFO kernel.cc:1022] Use fast generic engine


Model compiled.


Model: "random_forest_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (1560):
	song100_acousticness
	song100_danceability
	song100_energy
	song100_explicit
	song100_instrumentalness
	song100_key
	song100_liveness
	song100_loudness
	song100_mode
	song100_speechiness
	song100_tempo
	song100_time_signature
	song100_valence
	song101_acousticness
	song101_danceability
	song101_energy
	song101_explicit
	song101_instrumentalness
	song101_key
	song101_liveness
	song101_loudness
	song101_mode
	song101_speechiness
	song101_tempo
	song101_time_signature
	song101_valence
	song102_acousticness
	song102_danceability
	song102_energy
	song102_explicit
	song102_instrumentalness
	song102_key
	song102_liveness
	song102

In [20]:

evaluation = model.evaluate(test_ds, return_dict=True)

print(evaluation)
print()
print(f"MSE: {evaluation['mse']}")
print(f"RMSE: {np.sqrt(evaluation['mse'])}")

1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - mse: 0.0305 - cosine_similarity: 1.0000
{'loss': 0.0, 'mse': 0.030524836853146553, 'cosine_similarity': 1.0}

MSE: 0.030524836853146553
RMSE: 0.1747135851991669


In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

clf = linear_model.Lasso(alpha=0.1, random_state=42)
clf.fit(X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]), y_train[:,1])
yhat = clf.predict(X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]))
print('Best model MSE on test data = ', mean_squared_error(y_test[:,1], yhat))

Best model MSE on test data =  0.029280467354562415


In [21]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

clf = linear_model.Ridge(alpha=0.1, random_state=42)
clf.fit(X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]), y_train[:,1])
yhat = clf.predict(X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]))
print('Best model MSE on test data = ', mean_squared_error(y_test[:,1], yhat))

Best model MSE on test data =  0.06704051909451565
